#### Preliminaries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from Mean_Reversion.data_prep import fetch_data


#### Fetch Data

In [2]:
xom_df = fetch_data('XOM', '2017-11-30', '2024-09-18').drop(['Dividends', 'Stock Splits'], axis = 'columns')

In [3]:
xom_df

,Open,High,Low,Close,Volume
Date,,,,,
2017-11-30 00:00:00-05:00,59.767313,60.455963,59.738322,60.376225,18592700
2017-12-01 00:00:00-05:00,60.484978,60.985150,59.738340,60.499474,13779300
2017-12-04 00:00:00-05:00,60.318243,61.021389,60.166016,60.579205,12597700
2017-12-05 00:00:00-05:00,60.463235,60.724191,59.955808,60.086288,9357200
2017-12-06 00:00:00-05:00,60.057277,60.310988,59.593347,59.644089,13398900
...,...,...,...,...,...
2024-09-11 00:00:00-04:00,110.779999,110.949997,107.769997,109.720001,19571900
2024-09-12 00:00:00-04:00,109.900002,111.330002,109.760002,111.230003,15923300
2024-09-13 00:00:00-04:00,111.470001,112.470001,110.930000,111.150002,10035400


#### Apply Japanese Candlesticks

Use of this website:
    https://www.babypips.com/learn/forex/what-is-a-japanese-candlestick

- If close > open, white
- If open > close, black

white = 0

black = 1

In [4]:
def apply_jcandle(df):
    df.loc[df['Close'] > df['Open'], 'j_candle'] = 0
    df.loc[df['Close'] < df['Open'], 'j_candle'] = 1
    df.loc[df['j_candle'] == 0, 'high_wick_length'] = df['High'] - df['Close']
    df.loc[df['j_candle'] == 1, 'high_wick_length'] = df['High'] - df['Open']
    df.loc[df['j_candle'] == 0, 'low_wick_length'] = df['Close'] - df['Low']
    df.loc[df['j_candle'] == 1, 'low_wick_length'] = df['Open'] - df['Low']
    df['body_length'] = abs(df['Close'] - df['Open'])

    return df

In [5]:
xom_df = apply_jcandle(xom_df)

In [6]:
xom_df

,Open,High,Low,Close,Volume,j_candle,high_wick_length,low_wick_length,body_length
Date,,,,,,,,,
2017-11-30 00:00:00-05:00,59.767313,60.455963,59.738322,60.376225,18592700,0.0,0.079739,0.637903,0.608912
2017-12-01 00:00:00-05:00,60.484978,60.985150,59.738340,60.499474,13779300,0.0,0.485676,0.761133,0.014495
2017-12-04 00:00:00-05:00,60.318243,61.021389,60.166016,60.579205,12597700,0.0,0.442184,0.413188,0.260961
2017-12-05 00:00:00-05:00,60.463235,60.724191,59.955808,60.086288,9357200,1.0,0.260956,0.507428,0.376947
2017-12-06 00:00:00-05:00,60.057277,60.310988,59.593347,59.644089,13398900,1.0,0.253711,0.463930,0.413188
...,...,...,...,...,...,...,...,...,...
2024-09-11 00:00:00-04:00,110.779999,110.949997,107.769997,109.720001,19571900,1.0,0.169998,3.010002,1.059998
2024-09-12 00:00:00-04:00,109.900002,111.330002,109.760002,111.230003,15923300,0.0,0.099998,1.470001,1.330002
2024-09-13 00:00:00-04:00,111.470001,112.470001,110.930000,111.150002,10035400,1.0,1.000000,0.540001,0.320000


#### Visualization

In [7]:
import plotly.graph_objects as go

In [9]:
def visualize_candlestick(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
    return fig

In [10]:
fig = visualize_candlestick(xom_df)
fig.show()